In [1]:
!pip install python-docx pandas openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 4.5 MB/s eta 0:00:00


In [5]:
from google.colab import files


print('Sube la plantilla FR-EI-02 (Word)') # 👉 Aquí selecciona Plantilla_datos.xlsx y el Word
uploaded = files.upload()
FR_EI_02 = list(uploaded.keys())[0]

# Subir plantilla DATOS
print('Sube la plantilla DATOS')
uploaded = files.upload()
PLANTILLA_DATOS = list(uploaded.keys())[0]


Sube la plantilla FR-EI-02 (Word)


Saving FR-EI-02 ENTREGA DE EQUIPO A CONFORMIDAD Y CONDICIONES DE GARANTÍA (1).docx to FR-EI-02 ENTREGA DE EQUIPO A CONFORMIDAD Y CONDICIONES DE GARANTÍA (1) (1).docx
Sube la plantilla DATOS


Saving DATOS CLIENTES.xlsx to DATOS CLIENTES (1).xlsx


In [6]:
# Subir FORMATO
print('Sube FR-EI-04 HOJA DE VIDA DEL EQUIPO  FORMATO')
uploaded = files.upload()
FORMATO_HOJA_DE_VIDA = list(uploaded.keys())[0]

Sube FR-EI-04 HOJA DE VIDA DEL EQUIPO  FORMATO


Saving FR-EI-04 HOJA DE VIDA DEL EQUIPO - PROYECTO.xlsx to FR-EI-04 HOJA DE VIDA DEL EQUIPO - PROYECTO (1).xlsx


In [9]:
!pip install python-docx pandas openpyxl
from google.colab import files
import pandas as pd
from docx import Document
from openpyxl import load_workbook
import os, shutil, zipfile

# ==============================
# 1. Subir archivos base
# ==============================
#print("👉 Sube Plantilla_datos.xlsx")
#uploaded = files.upload()
#PLANTILLA_DATOS = list(uploaded.keys())[0]

#print("👉 Sube FR-EI-02 (Word)")
#uploaded = files.upload()
#FR_EI_02 = list(uploaded.keys())[0]

#print("👉 Sube FR-EI-04 (Excel)")
#uploaded = files.upload()
#FR_EI_04 = list(uploaded.keys())[0]

# ==============================
# 2. Leer los datos
# ==============================
df = pd.read_excel(PLANTILLA_DATOS)

# Crear carpeta de salida
output_dir = "Resultados_Finales"
os.makedirs(output_dir, exist_ok=True)

# ==============================
# 3. Iterar sobre cada fila
# ==============================
for index, row in df.iterrows():
    cliente = str(row["CLIENTE"]).strip().replace(" ", "_")
    equipo  = str(row["NOMBRE DEL EQUIPO"]).strip().replace(" ", "_")
    referencia = str(row["REFERENCIA"]).strip() if pd.notna(row["REFERENCIA"]) else ""
    serie      = str(row["SERIE"]).strip() if pd.notna(row["SERIE"]) else ""
    fecha_word = str(row["FECHA INSTALACION(WORD)"]).strip() if pd.notna(row["FECHA INSTALACION(WORD)"]) else ""

    # Fechas separadas
    dia  = str(row["DD"]).strip() if pd.notna(row["DD"]) else ""
    mes  = str(row["MM"]).strip() if pd.notna(row["MM"]) else ""
    anio = str(row["AA"]).strip() if pd.notna(row["AA"]) else ""

    entidad  = str(row["ENTIDAD"]).strip() if pd.notna(row["ENTIDAD"]) else ""
    ciudad   = str(row["CIUDAD"]).strip() if pd.notna(row["CIUDAD"]) else ""
    telefono = str(row["TELEFONO CLIENTE"]).strip() if pd.notna(row["TELEFONO CLIENTE"]) else ""
    #valor    = str(row["VALOR"]).strip() if pd.notna(row["VALOR"]) else ""

    # Carpeta temporal por cliente/equipo
    folder_name = f"{cliente}_{equipo}"
    folder_path = os.path.join(output_dir, folder_name)
    os.makedirs(folder_path, exist_ok=True)

    # ==============================
    # A) Generar Word
    # ==============================
    doc = Document(FR_EI_02)

    # Reemplazar cliente
    for p in doc.paragraphs:
        if "(Nombre cliente)" in p.text:
            p.text = p.text.replace("(Nombre cliente)", row["CLIENTE"])

    # Llenar la tabla
    table = doc.tables[0]
    table.cell(0,1).text = str(row["NOMBRE DEL EQUIPO"])
    table.cell(1,1).text = referencia
    table.cell(2,1).text = serie
    table.cell(3,1).text = fecha_word

    word_path = os.path.join(folder_path, f"Entrega_{cliente}_{equipo}.docx")
    doc.save(word_path)

    # ==============================
    # B) Generar Excel
    # ==============================
    wb = load_workbook(FORMATO_HOJA_DE_VIDA)
    ws = wb.active

    ws["D9"]  = row["NOMBRE DEL EQUIPO"]
    ws["V24"] = dia
    ws["X24"] = mes
    ws["Y24"] = anio
    ws["D22"]  = entidad
    ws["AE22"] = ciudad
    ws["AE24"] = telefono
    #ws["Y25"]  = valor
    ws["AD7"] = serie

    excel_path = os.path.join(folder_path, f"HojaVida_{cliente}_{equipo}.xlsx")
    wb.save(excel_path)

    # ==============================
    # C) Crear ZIP de cliente_equipo
    # ==============================
    zip_path = os.path.join(output_dir, f"{cliente}_{equipo}.zip")
    with zipfile.ZipFile(zip_path, "w") as zipf:
        for f in os.listdir(folder_path):
            zipf.write(os.path.join(folder_path, f), arcname=f)

    shutil.rmtree(folder_path)

print("✅ Proceso terminado. Los ZIP están en 'Resultados_Finales'")
shutil.make_archive("Resultados_Todos", "zip", "Resultados_Finales")
files.download("Resultados_Todos.zip")



/usr/local/lib/python3.12/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


✅ Proceso terminado. Los ZIP están en 'Resultados_Finales'


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>